In [1]:
print("ok!")

ok!


In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import pinecone
import pinecone 
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


c:\Users\fouzia\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [7]:
PINECONE_API_KEY = "pcsk_2Utdae_AHvuS9XVfHBxGxKVW3R7scQenoL8Lb1B86bJNqSJBa2F2FiujnjyLhw7pPs3qJu"
PINECONE_API_ENV = "us-east-1"

In [8]:
#Extract data from PDF

In [11]:

def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents
